# Print pokemons

api: https://github.com/rubenvereecken/pokemongo-api

In [ ]:
import print_pokemons as pp
from pogo.pokedex import pokedex

pokemon_types = pp.Types()

In [ ]:
keyfile = 'keyfile'
username, password, auth_type = pp.get_credentials_from_keyfile(keyfile)

In [ ]:
trainer = pp.get_trainer(username, password, auth_type)

In [ ]:
inventory = trainer.session.getInventory()
bag = pp.get_pokemons(trainer)

## Count pokemons and possible evolves

In [ ]:
from collections import Counter

def print_most_common(bag, limit=None, curr_evo_only=False, exclude=None):
    if exclude is None:
        exclude = list()
    counter = Counter([(x['name'], x['id']) for x in bag if pokedex.evolves.get(x['id'], 0) > 0])

    print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format('Name', 'Need', 'Candies', 'NumHave', 'Can Evo.', 'Curr. Evo.'))
    print('-'*70)

    total_possible = 0
    total_current = 0
    if limit is None:
        most_common = sorted(counter.most_common())
    else:
        most_common = sorted(counter.most_common(limit))

    for item, count in most_common:
        family = pokedex.families[item[1]]

        n_candies = inventory.candies.get(item[1], 0)
        need = pokedex.evolves.get(item[1], 0)

        possible = int(n_candies / need)
        curr = min(possible, count)
        if curr_evo_only and curr == 0:
            continue
        if item[0] in exclude:
            continue
        print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format(item[0], need, n_candies, count, possible, curr))

        total_possible += possible
        total_current += curr

    print('\nTotal possible evolutions: {}'.format(total_possible))
    print('Current total evolutions: {}'.format(total_current))

## Examples of pokemon filtering

### Filter by Type

In [ ]:
lst = [x for x in bag if pokemon_types.WATER in pokemon_types[x['id']]]

### All from list of types

In [ ]:
lst = [x for x in bag if any(i in [pokemon_types.FAIRY, pokemon_types.NORMAL] for i in pokemon_types[x['id']])]

### Have all this types

In [ ]:
lst = [x for x in bag if all(i in pokemon_types[x['id']] for i in [pokemon_types.FAIRY, pokemon_types.NORMAL])]

### Print pokemon types

In [ ]:
for t in pokemon_types[pokedex.MR_MIME]:
    print(pokemon_types.ALL_TYPES[t])

## Snippets

In [ ]:
# Use bag to get all pokemons
lst = bag

In [ ]:
pp.print_pokemons_by_iv(lst, segments=(93, 85, 82, 75, 0))

In [ ]:
pp.print_pokemons_by_number(lst)

In [ ]:
print_most_common(lst, curr_evo_only=True, exclude=('DRATINI', 'CHARMANDER'))